In [1]:
#!/usr/bin/env python

import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display
import csv

#---------------------- ----------------------------- -------------
# Prevent chrome browser from opening while browsing 
display = Display(visible=0, size=(800, 600))
display.start()
# Set up webdriver with chrome engine 
s=Service("/home/revo/chromedriver/chromedriver")
driver = webdriver.Chrome(service=s)
#driver = webdriver.Chrome("/home/revo/chromedriver/chromedriver")

#----------------------------------------------------------------------------

# IIoT devices manufacturing companies
iiot_list = ('Linx', 'Johnson', 'GridConnect', 'Impinj')

# ICS devices manufacturing companies
ics_list = ('GE Digital', 'Bosch', 'Siemens', 'Mitsubishi', 'Schneider', 'ABB', 'Schweitzer', 'General Electric', 'Wind River')

#test
test_list = ["Asus RT-N10+ D1 router", "Huawei Quidway switches", "SerVision HVG Video Gateway", 
            "D-Link", "DIR-645 Wired/Wireless Router", "Zhone GPON 2520", "Dell SonicWALL Secure Remote Access",
            "AirLive BU-2015", "ASUS RT-G32 routers", "AirTies", "Alcatel-Lucent OmniSwitch", "ANTlabs InnGate",
            "Grandstream GXV3611_HD camera", "Actiontec GT784WN modems"]

# IoT devices manufacturing companies
iot_list = ('Amazon Echo', 'Google Home', 'Verizon', 'digi', 'Particle', 'SmartCare', 'Edimax', 'ZigBee', 'Z-Wave', 'E2 Max', 'E7 Beacon', 'P1 Plus', 'August Doorbell', 'August Smart', 'Kuri Mobile', 'Belkin WeMo', 'Footbot Air', 'Nest Smoke', 'Nest T3021US', 'Philips Hue', 'Bitdefender BOX', 'Ring Doorbell', 'WeMo Insight', 'Logitech Harmony', 'Particle Photon', 'NETGEAR Orbi', 'Canary', 'Cinder', 'Nespresso Prodigio', 'Netatmo', 'Samsung SmartThings', 'Sonos', 'Wink', 'Apple Watch', 'Garmin Forerunner', 'Peloton', 'Google Cardboard', 'Samsung Gear')



embedded_systems = iiot_list+ics_list+iot_list
embedded_iot = iot_list
embedded_ics = ics_list
embedded_iiot = iiot_list

cve_id = "" # Points to the CVE where the word is found
foundCves = [] # Stores the found CVEs using the word lists

# The delimiter of CVEs in cve.txt
delim = '======================================================' 



'''non_embedded = open("non_embedded.txt", "w+")
array = []
with open("filtredCVEs.txt", "r") as fcve:
    ff = fcve.readlines()
    for i in range(len(ff)):
        array.append(ff[i].strip())
    for entry in uniqFoundCves:
        if entry not in array:
            non_embedded.write(entry+'\n')'''




'non_embedded = open("non_embedded.txt", "w+")\narray = []\nwith open("filtredCVEs.txt", "r") as fcve:\n    ff = fcve.readlines()\n    for i in range(len(ff)):\n        array.append(ff[i].strip())\n    for entry in uniqFoundCves:\n        if entry not in array:\n            non_embedded.write(entry+\'\n\')'

In [12]:
#---------------------------------------------------------------
with open("cve.txt", "r") as file:
    lines = file.readlines()
    for i in range(0, len(lines)):
        if re.search(delim, lines[i]):
            cve_id = lines[i+1]
        for word in test_list:
            if re.search("Reference:", lines[i]):
                break                   
            elif re.search(r'\b'+word+r'\b', lines[i]):
                # Store CVES in foundCves array
                foundCves.append(cve_id[6:-1])
                break
uniqFoundCves = list(dict.fromkeys(foundCves))
print(len(uniqFoundCves))


2776


In [11]:
#======================================================================

cve_cwe_array = {} # Stors all exist {'CVE': 'CWE'} in the json file
cwes = [] # Stores the CWE of only CVEs in the uniqFoundCves list
non_exist_foundCves = [] # Stors non-defined in NVD CWEs
for year in range(2015, 2022):
    with open("Json_database/nvdcve-1.1-"+str(year)+".json", "r") as f:
        data = json.load(f)
        for i in data['CVE_Items']:
            for j in range(0, len(i['cve']['problemtype']['problemtype_data'])):
                for k in range(0, len(i['cve']['problemtype']['problemtype_data'][j]['description'])):
                    # 'CVE':'CWE'
                    if i['cve']['CVE_data_meta']['ID'] in uniqFoundCves:
                        cve_cwe_array[i['cve']['CVE_data_meta']['ID']] = i['cve']['problemtype']['problemtype_data'][j]['description'][k]['value']

print(cve_cwe_array)

{'CVE-2015-0982': 'CWE-119', 'CVE-2015-0996': 'CWE-200', 'CVE-2015-0997': 'CWE-200', 'CVE-2015-0998': 'CWE-200', 'CVE-2015-0999': 'CWE-200', 'CVE-2015-1009': 'CWE-200', 'CVE-2015-1014': 'CWE-427', 'CVE-2015-1048': 'NVD-CWE-Other', 'CVE-2015-1049': 'CWE-20', 'CVE-2015-1355': 'CWE-310', 'CVE-2015-1356': 'CWE-264', 'CVE-2015-1357': 'CWE-200', 'CVE-2015-1358': 'CWE-310', 'CVE-2015-1448': 'CWE-264', 'CVE-2015-1449': 'CWE-119', 'CVE-2015-1594': 'NVD-CWE-Other', 'CVE-2015-1595': 'CWE-200', 'CVE-2015-1596': 'CWE-310', 'CVE-2015-1597': 'CWE-94', 'CVE-2015-1598': 'CWE-200', 'CVE-2015-1599': 'CWE-264', 'CVE-2015-1600': 'CWE-200', 'CVE-2015-1601': 'CWE-254', 'CVE-2015-1602': 'CWE-200', 'CVE-2015-1628': 'CWE-79', 'CVE-2015-1851': 'CWE-200', 'CVE-2015-2177': 'CWE-20', 'CVE-2015-2822': 'CWE-20', 'CVE-2015-2823': 'CWE-287', 'CVE-2015-3610': 'CWE-310', 'CVE-2015-3938': 'CWE-399', 'CVE-2015-3940': 'CWE-22', 'CVE-2015-3962': 'CWE-200', 'CVE-2015-3963': 'CWE-330', 'CVE-2015-3974': 'CWE-255', 'CVE-2015-397

In [5]:
non_embedded = open("non_embedded.txt", "w+")
array = []
with open("filtredCVEs.txt", "r") as fcve:
    ff = fcve.readlines()
    for i in range(len(ff)):
        array.append(ff[i].strip())
    for entry in cvess:
        if entry not in array:
            non_embedded.write(entry+'\n')

In [4]:
#!usr/bin/python



'''for j in cvess:
    cvelist.writelines([j, '\n'])'''
#=====================================================================================

    #for k in exist:
        #print(table[k])

#=====================================================================================

'''table = {} # Stors all exist {'CVE': 'CWE'} in the json file
cwes = [] # Stores the CWE of only CVEs in the cvess list
non_exist_cves = [] # Stors non-defined in NVD CWEs

with open("nvdcve-1.1-2015.json", "r") as f:
    data = json.load(f)
    for i in data['CVE_Items']:
        for j in range(0, len(i['cve']['problemtype']['problemtype_data'])):
            for k in range(0, len(i['cve']['problemtype']['problemtype_data'][j]['description'])):
                # 'CVE':'CWE'
                 table[i['cve']['CVE_data_meta']['ID']] = i['cve']['problemtype']['problemtype_data'][j]['description'][k]['value']
    #print(len(table))
    for cve in table:
        if cve in cvess:
            exist_cves.append(cve)
    print(len(exist_cves))

    noninfo = 0
    for cve in cvess:
        try:
            cwes.append(table[cve])
            if re.search('NVD', table[cve]):
                noninfo += 1
        except KeyError:
            non_exist_cves.append(cve)
    #print(len(cwes))
    #print(noninfo)
    #print(len(non_exist_cves))

#=======================================================================================================================================

with open('cve_cwe.csv', 'w+') as f:
    write =  csv.writer(f)
    for j in cvess:
        try:
            driver.get("https://cwe.mitre.org/data/definitions/"+table[j][4:]+".html")
            content = driver.page_source
            soup = (content)
            for div in soup.findAll('div', attrs={'style':'overflow:auto;'}):
                write.writerow([j, div.h2.string])
        except KeyError:BeautifulSoup
            continue

f.close()
'''
# Let store the CVEs in an html file to make the parsing more easier
link = "https://nvd.nist.gov/vuln/detail/"

arraylist = []
htmlfile = open("testhtml.html", "w+")
for entry in cvess:
    arraylist.append(entry)

for value in arraylist:
     htmlfile.write('<a href='+link+value+'>'+value+'</a></br>')

file.close()

#==========================================================================================================================================

In [7]:
import pandas as pd
from pandas.plotting import scatter_matrix
#import seaborn as sns
#import matplotlib.pyplot as plt

datalist = []
data = pd.read_csv("cve_cwe.csv")
data.describe()
cweplot = data.groupby('CWE')['CVE'].count()

#sns.factorplot(x="CWE", y ="Num", data=data, kind="bar", size=3)
#plt.show()
#%matplotlib inline
#cweplot.plot(kind='bar', stacked=True, title="The title of my graph")

#scatter_matrix(data)
#data['CWE'].value_counts()
#uniques = pd.unique(data['CWE'])
#with open("cwedata.csv", "w+") as fcsv:
#    writef =  csv.writer(fcsv)
#    for i in uniques:
#        writef.writerow([data.groupby('CWE')['CVE'].count()[i], i])
#print (summ)

KeyError: 'CWE'

In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome("/home/revo/chrome-driver/chromedriver")

# List to store CWE-Names
cwe_names = []

driver.get("https://cwe.mitre.org/data/definitions/259.html")

content = driver.page_source
soup = BeautifulSoup(content)
for div in soup.findAll('div', attrs={'style':'overflow:auto;'}):
    print(div.h2.string)


CWE-259: Use of Hard-coded Password


In [61]:
import csv
import re

iotList = []
icsList = []
common  = []

iot_f = open("iot_cwedata.csv", "r")
ics_f = open("ics_cwedata.csv", "r")
diff_iot = open("iot_diff", "w+")
diff_ics = open("ics_diff", "w+")
iot = csv.reader(iot_f)
ics = csv.reader(ics_f)

for i in ics:
    icsList.append(i[1])
for j in iot:
    iotList.append(j[1])

for k in list(set(iotList).difference(icsList)):
    diff_iot.writelines([k, '\n'])